In [1]:
import girder_client
import glob,os,sys

In [2]:
localDirToSync = "/home/dagutman/data/pTDP-43 IHC/"  ## MAke sure theres a forward slash at the end for both
tgtResourcePath = "/JCV_ImageAnalysis/SammyLee_ImageSet/" ### the FIRST root is a collection make sure both this and localDirtoSync have a /

gc = girder_client.GirderClient(apiUrl="http://computablebrain.emory.edu:8080/api/v1")
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


class LinePrinter():
    """
    Print things to stdout on one line dynamically
    """
    def __init__(self,data):
        sys.stdout.write("\r\x1b[K"+data.__str__())
        sys.stdout.flush()
        
        
def girderMakeDirs( resourcePath ):
    ### Let's see if the entire resource path exists, if it doesn't I'll recurse and create it
    rp = gc.get("resource/lookup?path=/collection%s&test=True" % resourcePath)
    if rp:
        return rp ## Found the resource path, so I'll create it..
    
    rsrcParts = resourcePath.split("/")
    ### Now make sure every part exists...
    collectionName = rsrcParts[1]
    ### To start, I need the collection ID, then this becomes the folder ID for subsequent calls
    
    resourceLookupPath = "resource/lookup?path=/collection/%s"%collectionName
    collInfo  = gc.get(resourceLookupPath)
    
    ### The collection is the starting point so I am starting at the 2nd element in the resourcePath
    parentType='collection' ## On the first pass, the parentType is collection ,it then becomes a folder
    parentId = collInfo['_id']
    for i in range(2,len(rsrcParts)):
        #print( "/".join(rsrcParts[2:i]))
        fldrInfo = gc.createFolder(parentId=parentId, name=rsrcParts[i], parentType=parentType,reuseExisting=True)
        parentType='folder'
        parentId=fldrInfo['_id']
    
    return(fldrInfo)

In [6]:
grdrFolderIdCache = {}
dirsProcessed = filesProcessed = skippedFiles = uploadedFiles = 0

for root, dirs, files in os.walk(localDirToSync, topdown=False):
    dirsProcessed +=1
    
    for name in files:
        filesProcessed +=1
        rsrcPath = root.replace(localDirToSync,tgtResourcePath)
        localFileName = os.path.join(root, name)

        if rsrcPath not in grdrFolderIdCache:
            grdrFldr = girderMakeDirs(rsrcPath)            
            grdrFolderIdCache[rsrcPath] = grdrFldr
            ### May want to add the itemCache as well... will think about it
        ## Now that I know there's a folder, I am going to upload the item to it..
        
        ### So could just search for this item and see if it exists...
        
        ## CHECK IF THE ITEM / FILE HAS ALREADY BEEN UPLOADED
        itemsInFolder = list(gc.listItem( grdrFolderIdCache[rsrcPath]['_id'],name=name))
        if len(itemsInFolder) > 0:
            #print("Item already uploaded:",name,"To RsrcPath:",rsrcPath)
            skippedFiles+=1
        else:
            itemData = gc.uploadFileToFolder(grdrFolderIdCache[rsrcPath]['_id'], localFileName, filename=name)
            uploadedFiles+=1
            
        opt = "Processed %d dirs and %d total files; %d just uploaded; %d skipped" % ( dirsProcessed,filesProcessed,
                                                                                      uploadedFiles,skippedFiles)
        LinePrinter(opt)


Processed 21 dirs and 203 total files; 21 just uploaded; 182 skipped

KeyboardInterrupt: 

In [ ]:
##So just search the resource path for the item...
# IN case of a LEAFED folder I can upload multiple files to an item..
# NEED To think about indexing vs 
## so the itemData line would change to import from upload... to think about
        
                  ## So for TIF images it automatically createa a large image, but for JPGs or PNGs I may want to add this
            #         try:
            #             gc.post("item/%s/tiles" % itemData['itemId'])
            #         except:
            #             print "BEATS ME"
            #         gc.addMetadataToItem(itemData['itemId'],{"XTK": {"type":"volume"}})

  


In [ ]:
        
#     for name in dirs:
#         #print(os.path.join(root, name))
#         ### Creating Subdrectories..
        
#         ## Need to convert the local file path to a girder resource path
#         rsrcPath = root.replace(localDirToSync,tgtResourcePath)
#         girderMakeDirs(rsrcPath)